Import Statements

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random 
from pprint import pprint

Load and Prepare Data

In [5]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_threshold = 15

    for column in df.columns:
        unique_values = df[column].unique()
        example_value = unique_values[0]

        if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_threshold):
            feature_types.append("Discrete")
        else:
            feature_types.append("Continuous")
    
    return feature_types

In [6]:
df = pd.read_csv('census-income.data.csv')
global FEATURE_TYPES
FEATURE_TYPES = determine_type_of_feature(df)




In [7]:
#Checking which all columns have missing values
def missing_values(df):
    c=0
    for i in df.columns:
        x,count_missing = np.unique(df[i].eq('?'),return_counts=True)
        if len(x)==2:
            print(i,"-",FEATURE_TYPES[c],"-",count_missing[1])
        # elif len(x)==1 and x[0] == False:
        #     print(i,"-",FEATURE_TYPES[c],"- 0",)
        c+=1

Random Splitting Of Data

In [8]:
def train_test_split(df,test_size):
    #checks whether the test_size is a proportion of the total number of samples
    if isinstance(test_size,float): 
        test_size = round(test_size*len(df))

    #store random samples in the test and training data
    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k = test_size)

    #random data points from sample sent to test and training data
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)

    return train_df , test_df

Data Pure?


In [9]:
def check_purity(data):
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) ==  1:
        return True    
    else:
        return False 

Classify Data

In [10]:
def classify_data(data): 
    label_column = data[:,-1]
    unique_classes , counts_unique_classes = np.unique(label_column,return_counts = True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification 

In [11]:
def get_potential_splits(data):
    potential_splits = {}
    rows , n_columns = data.shape

    for column_index in range(n_columns-1):
        values = data[:,column_index]
        unique_values = np.unique(values)
        
        
        potential_splits[column_index] = unique_values
            
    return potential_splits

In [12]:
# potential_splits = get_potential_splits(train_df.values)

# sns.lmplot(data = train_df, x = "petal_width", y = "petal_length",hue = "label", fit_reg = False,height=6,aspect = 1.5)

# # plt.vlines(x = potential_splits[3], ymin = 1, ymax =7)
# # plt.hlines(y=potential_splits[2],xmin=0,xmax =2.5)

Split Data

In [13]:
def split_data(data, split_column,split_value):
    split_column_values = data[:,split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    
    if type_of_feature == "Continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values > split_value]
    
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]

    return data_below , data_above

In [14]:
# split_value = 0.8
# plotting_df = pd.DataFrame(data,columns = df.columns)
# sns.lmplot(data = plotting_df, x = "petal_width", y = "petal_length", fit_reg=False)
# plt.vlines(x = split_value,ymin = 1, ymax =7)

In [15]:
def calculate_entropy(data):
    label_column = data[:,-1]
    rows , counts = np.unique(label_column , return_counts =True)

    probabilities = counts / counts.sum()
    entropy = np.sum(probabilities*-np.log2(probabilities))

    return entropy

In [16]:
def calculate_overall_entropy(data_below, data_above):
    
    n_data_points = len(data_below) + len(data_above)

    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points

    overall_entropy = (p_data_below*calculate_entropy(data_below) + p_data_above*calculate_entropy(data_above))

    return overall_entropy

In [17]:
def determine_best_splits(data, potential_splits):

    overall_entropy = 999
    
    for column_index in potential_splits:
        # print(COLUMN_HEADERS[column_index],"-",len(np.unique(data[:, column_index])))
        for value in potential_splits[column_index]:
            
            data_below, data_above = split_data(data,split_column = column_index,split_value = value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

Determine Whether Column is Discrete or Continuous


In [18]:
def decision_tree_algorithm(df , counter =0, min_samples=5, max_depth = 5):
    # print(counter)
    #Data Preparation
    if counter == 0:
        global COLUMN_HEADERS 
        global FEATURE_TYPES 
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df

    #Base Case 
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification 
    
    #Recursive case
    else:
        counter = counter + 1
        
        #helper functions
        potential_splits = get_potential_splits(data)        
        split_column,split_value = determine_best_splits(data,potential_splits)
        data_below,data_above = split_data(data,split_column,split_value)

        #checking for empty data

        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification

        #instantiating the sub-trees
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "Continuous":
            question = "{} <= {}".format(feature_name,split_value)

        else:
            question = "{} = {}".format(feature_name,split_value)

        sub_tree = {question:[]}

        #find answers (here recursion is used)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples , max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples , max_depth)

        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)

        return sub_tree




In [19]:
def classify_example(example,tree):
    question = list(tree.keys())[0]
    feature_name , comparison_operator, value = question.split()

    #Asking the question to the example
    if comparison_operator == "<=":

        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    #base Case
    if not isinstance(answer,dict):
        return answer

    #recursive part
    else:
        remaining_tree = answer
        return classify_example(example,remaining_tree)

In [20]:
def calculate_accuracy(df,tree):

    df["Classification"] = df.apply(classify_example,axis=1,args = (tree,))
    df["Classification_Correct"] = df.Classification == df.label

    accuracy = df.Classification_Correct.mean()

    return accuracy

In [21]:
def make_predictions(df, tree):
    
    if len(df) != 0:
        predictions = df.apply(classify_example, args=(tree,), axis=1)
    else:
        # "df.apply()"" with empty dataframe returns an empty dataframe,
        # but "predictions" should be a series instead
        predictions = pd.Series()
        
    return predictions

In [22]:
def filter_df(df,question):
    feature,comparison_operator,value = question.split()

    if (comparison_operator) == "<=":
        df_yes = df[df[feature]<= float(value)]
        df_no = df[df[feature] > float(value)]
    
    else:
        df_yes = df[df[feature].astype(str) == value]
        df_no = df[df[feature].astype(str) != value]
        

    return df_yes,df_no

In [23]:
def pruning_result(tree, df_train, df_val):
    leaf = df_train.label.value_counts().index[0]
    errors_leaf = sum(df_val.label != leaf) #here we can find the accuracy also :D
    errors_decision_node = sum(df_val.label != make_predictions(df_val,tree))

    if errors_leaf <= errors_decision_node:
            return leaf
    else:
            return tree

In [24]:
def post_pruning(tree , df_train , df_val):
    question = list(tree.keys())[0]
    yes_answer , no_answer = tree[question]

    #base case
    if not isinstance(yes_answer,dict) and not isinstance(no_answer,dict):
        return pruning_result(tree , df_train , df_val)
    
    else: #Recursive case
        df_train_yes, df_train_no = filter_df(df_train , question)
        df_val_yes, df_val_no = filter_df(df_val , question)

        if isinstance(yes_answer,dict):
            yes_answer = post_pruning(yes_answer,df_train_yes,df_val_yes)

        if isinstance(no_answer,dict):
            no_answer = post_pruning(no_answer,df_train_no,df_val_no)
    
    tree = {question : [yes_answer,no_answer]}

    return pruning_result(tree , df_train , df_val)

In [25]:
def count_nodes(tree):
    if isinstance(tree, dict):
        count = 1  # Start with 1 for the current node
        for value in tree.values():
            count += count_nodes(value)  # Recursively count nodes in child dictionaries
        return count
    elif isinstance(tree, list):
        count = 0
        for node in tree:
            count += count_nodes(node)  # Recursively count nodes in child dictionaries
        return count
    else:
        return 1  # Reached a leaf node

In [26]:
def count_depth(tree):
    if isinstance(tree, dict):
        if not tree:  # Base case: empty dictionary
            return 1
        else:
            return 1 + max(count_depth(value) for value in tree.values())
    elif isinstance(tree, list):
        if not tree:  # Base case: empty list
            return 1
        else:
            return 1 + max(count_depth(item) for item in tree)
    else:
        return 0  # Base case: leaf node or invalid input

In [27]:
df1 = pd.read_csv("census-income.data.csv")
df2 = pd.read_csv("census-income.test.csv")


In [28]:
df2['label'] = df2['label'].str.replace('.', '')

In [29]:
missing_values(df1)

workclass - Discrete - 1836
occupation - Discrete - 1843
native_countr - Discrete - 583


In [30]:
def fill_values(df):
    #Since all columns are having the discrete value. We replace it with the missing values with the modes, i.e. the highest appearing value
    mode_workclass = df.workclass.mode()[0]
    mode_occupation = df.occupation.mode()[0]
    mode_native_countr = df.native_countr.mode()[0]

    #Filling the train and test data with the modes of missing values as they are discrete
    df.workclass = df.workclass.replace('?', mode_workclass)
    df.occupation = df.occupation.replace('?', mode_occupation)
    df.native_countr = df.native_countr.replace('?', mode_native_countr)
    missing_values(df)#No missing values
    return df

In [31]:
df1 = fill_values(df1)
df2 = fill_values(df2)

In [32]:
df_combined = pd.concat([df1,df2],ignore_index=True)

In [33]:
missing_values(df_combined)

In [34]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education_num   48842 non-null  int64 
 5   marital_status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital_gain    48842 non-null  int64 
 11  capital_loss    48842 non-null  int64 
 12  hours_per_week  48842 non-null  int64 
 13  native_countr   48842 non-null  object
 14  label           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [35]:
df_train , df_test = train_test_split(df_combined , 0.33 )

In [36]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32724 entries, 2 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32724 non-null  int64 
 1   workclass       32724 non-null  object
 2   fnlwgt          32724 non-null  int64 
 3   education       32724 non-null  object
 4   education_num   32724 non-null  int64 
 5   marital_status  32724 non-null  object
 6   occupation      32724 non-null  object
 7   relationship    32724 non-null  object
 8   race            32724 non-null  object
 9   sex             32724 non-null  object
 10  capital_gain    32724 non-null  int64 
 11  capital_loss    32724 non-null  int64 
 12  hours_per_week  32724 non-null  int64 
 13  native_countr   32724 non-null  object
 14  label           32724 non-null  object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB


In [37]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16118 entries, 19002 to 45360
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             16118 non-null  int64 
 1   workclass       16118 non-null  object
 2   fnlwgt          16118 non-null  int64 
 3   education       16118 non-null  object
 4   education_num   16118 non-null  int64 
 5   marital_status  16118 non-null  object
 6   occupation      16118 non-null  object
 7   relationship    16118 non-null  object
 8   race            16118 non-null  object
 9   sex             16118 non-null  object
 10  capital_gain    16118 non-null  int64 
 11  capital_loss    16118 non-null  int64 
 12  hours_per_week  16118 non-null  int64 
 13  native_countr   16118 non-null  object
 14  label           16118 non-null  object
dtypes: int64(6), object(9)
memory usage: 2.0+ MB


In [ ]:
tree_combined = decision_tree_algorithm(df_train,max_depth = 40)

In [41]:
tree = tree_combined

In [42]:
tree

{'marital_status = Married-civ-spouse': [{'education_num <= 11': [{'capital_gain <= 5013': [{'education_num <= 8': [{'hours_per_week <= 40': [{'education_num <= 5': [{'hours_per_week <= 24': ['<=50K',
              {'native_countr = Yugoslavia': ['>50K',
                {'workclass = Self-emp-inc': [{'native_countr = Mexico': ['>50K',
                    {'occupation = Machine-op-inspct': ['>50K', '<=50K']}]},
                  {'occupation = Handlers-cleaners': ['<=50K',
                    {'occupation = Prof-specialty': ['<=50K',
                      {'relationship = Wife': [{'education = 5th-6th': [{'fnlwgt <= 175847': [{'hours_per_week <= 35': ['<=50K',
                              '>50K']},
                            '<=50K']},
                          '<=50K']},
                        {'fnlwgt <= 144594': [{'occupation = Machine-op-inspct': [{'age <= 39': [{'fnlwgt <= 90273': ['>50K',
                                '<=50K']},
                              '<=50K']},
      

In [43]:
count_depth(tree)

80

In [44]:
count_nodes(tree)

6533

In [45]:
pruned_tree = post_pruning(tree ,df_train,df_test)

In [46]:
pruned_tree

{'marital_status = Married-civ-spouse': [{'education_num <= 11': [{'capital_gain <= 5013': [{'education_num <= 8': [{'hours_per_week <= 40': [{'education_num <= 5': [{'hours_per_week <= 24': ['<=50K',
              {'native_countr = Yugoslavia': ['>50K',
                {'workclass = Self-emp-inc': ['<=50K',
                  {'occupation = Handlers-cleaners': ['<=50K',
                    {'occupation = Prof-specialty': ['<=50K',
                      {'relationship = Wife': ['<=50K',
                        {'fnlwgt <= 144594': ['<=50K',
                          {'occupation = Exec-managerial': ['<=50K',
                            {'fnlwgt <= 155106': ['<=50K',
                              {'capital_loss <= 1672': ['<=50K',
                                {'capital_loss <= 1902': ['>50K',
                                  '<=50K']}]}]}]}]}]}]}]}]}]}]},
            {'age <= 37': ['<=50K',
              {'capital_loss <= 1740': [{'age <= 66': [{'age <= 38': ['<=50K',
               

In [47]:
count_nodes(pruned_tree)

1557

In [48]:
count_depth(pruned_tree)

80

In [49]:
def acc_on_sets(tree,train_df,test_df):
    accuracy_train = calculate_accuracy(train_df,tree)
    accuracy_test = calculate_accuracy(test_df,tree)

    print("Training Set Accuracy:",accuracy_train)
    print("Test Set Accuracy:",accuracy_test)


In [50]:
acc_on_sets(tree , df_train, df_test)#combined tree

Training Set Accuracy: 0.9706942916513873
Test Set Accuracy: 0.8236753939694751


In [51]:
acc_on_sets(pruned_tree,df_train,df_test)

Training Set Accuracy: 0.8990343478792324
Test Set Accuracy: 0.8733713860280432
